In [1]:
%matplotlib inline
import csv
import requests
from requests.exceptions import *
import psycopg2
import json
import sqlite3
import time
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
from lxml import html
from bs4 import BeautifulSoup
from datetime import date,datetime

In [2]:
# Read items data

items_df = pd.read_csv("items.csv")
items_df = items_df.sort_values('item_id').reset_index(drop=True)

items_df

,item_id,link,title,image,category1_id,category1_name,category2_id,category2_name,category3_id,category3_name,...,brand,price,likes,description,keyword1,keyword2,keyword3,keyword4,keyword5,keyword6
0,10517,http://www.polyvore.com/set_insect_tac_pins/th...,Set of 3 insect tac pins,https://ak1.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,60,Jewelry,67,Brooches,...,NaN,150 TWD,590,Shaped of silver filigree Pegasus flies throug...,Gold jewelry,Pin jewelry,NaN,NaN,NaN,NaN
1,12045,http://www.polyvore.com/shoes_accessories_pedr...,"Shoes and Accessories by Pedro García, Renè Ca...",https://ak1.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,41,Shoes,46,Sandals,...,NaN,NaN,489,Glossy studs and pretty bows add feminine char...,NaN,NaN,NaN,NaN,NaN,NaN
2,14024,http://www.polyvore.com/designer_jewellery_jew...,Designer Jewellery Jewelry Fashion Costume Vin...,https://ak2.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,318,Handbags,38,Clutches,...,Swarovski,NaN,523,Modern shape meets vintage whimsy in our small...,Vintage purses,Red hand bags,Man bag,Handbag purse,Crystal clutches,Purse clutches
3,36366,http://www.polyvore.com/cgi/thing?id=36366,- DAY Birger et Mikkelsen - Polyvore,https://ak1.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,11,Tops,19,Sweaters,...,DAY+Birger+et+Mikkelsen,NaN,39,Make a youthful statement in this camel wool p...,DAY Birger et Mikkelsen,Knitwear sweater,NaN,NaN,NaN,NaN
4,74666,http://www.polyvore.com/bamboo_hoops_earrings/...,"3.5"" Bamboo Hoops Earrings",https://ak2.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,60,Jewelry,64,Earrings,...,NaN,180 TWD,3502,Posts about hoop earrings written by Rachel,Bamboo earrings,Gold jewellery,Hoop earrings,Earring jewelry,NaN,NaN
5,78059,http://www.polyvore.com/0711808102_scarlett_to...,0711808102 SCARLETT TOP WHT Product,https://ak2.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,2,Clothing,0,NaN,...,NaN,NaN,1226,An energetic and innovative aesthetic translat...,NaN,NaN,NaN,NaN,NaN,NaN
6,98959,http://www.polyvore.com/ae_strappy_tank_americ...,AE Strappy Tank - American Eagle Outfitters,https://ak1.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,11,Tops,104,Tank Tops,...,American+Eagle+Outfitters,NaN,5934,Yellow sleeveless silk and wool top by Prada.,American Eagle Outfitters,Spaghetti-strap tank tops,Strappy top,Shirt top,NaN,NaN
7,99084,http://www.polyvore.com/sterling_silver_slythe...,Sterling Silver Slytherin House Ring by Noble ...,https://ak1.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,60,Jewelry,65,Rings,...,NaN,1430 TWD,221,Lend after-hours ensembles a note of Parisian ...,NaN,NaN,NaN,NaN,NaN,NaN
8,105737,http://www.polyvore.com/mytheresa.com_whats_ne...,mytheresa.com . what's new . KURZER A-LINIEN M...,https://ak2.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,2,Clothing,23,Outerwear,...,NaN,NaN,2303,Add some Parisian flair courtesy of this warm ...,NaN,NaN,NaN,NaN,NaN,NaN
9,106678,http://www.polyvore.com/black_red_leather_ring...,Black And Red Leather Ring Collar,https://ak1.polyvoreimg.com/cgi/img-thing/size...,1,Women's Fashion,60,Jewelry,62,Necklaces,...,NaN,480 TWD,1627,Black And Red Leather Ring Collar: This .75 in...,Collar necklaces,Charm necklaces,Choker necklaces,Shiny charm,Metal choker,Wide choker


In [3]:
len(items_df)

355169

In [4]:
day = datetime.now().strftime("%Y%m%d") 
column_name = 'likes_' + str(day)
print column_name

likes_20180103


In [8]:
# Connect to postgres database
conn = psycopg2.connect(database="fashion", user="michellechen", password="", host="127.0.0.1", port="5432")

print "Opened database successfully"

# Connection cursor to database
cur = conn.cursor()

cur.execute('''ALTER TABLE items ADD ''' + column_name + ''' INT;''')

conn.commit() 

Opened database successfully


In [9]:
# Headers

head = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.65 Safari/537.36'}

In [33]:
i = 40222
while i < len(items_df):
    
    item_id = items_df.loc[i][0]
    item_link = items_df.loc[i][1]
    item_likes = 0
    
    print i, item_id
    
    # Get request response
    while True:
        try:
            res = requests.get(item_link, headers = head)
        except RequestException:
            print "Item Request Connection Error!"
            print "Delay 1 minutes!"
            # Delay 60 seconds
            time.sleep(60) 
        else:
            break

    # If item page is not found
    if res.status_code == 404:
        # Skip to next outfit
        print "HTTP Response Error " + str(res.status_code)
        print "Item Not Found!"
        
    else:    
        # When http response error
        while res.status_code != 200:
            # Print error code
            print "HTTP Response Error " + str(res.status_code)
            # Delay 5 seconds
            time.sleep(5) 
            # Requests again
            res = requests.get(item_link, headers = head) 
            if res.status_code == 200:
                break

        # Parse html with BeautifulSoup
        soup = BeautifulSoup(res.text, "html.parser")

        # Item likes
        item_likes = int(soup.find_all('meta', {'property': 'polyvore:saves'})[0]['content'])
        
        # Update likes count in database
        cur.execute( '''UPDATE items 
                        SET ''' + column_name + ''' = ''' + str(item_likes) + '''
                        WHERE item_id = ''' + str(item_id) + ''';''')
                
        conn.commit()
        
    i = i + 1

37899 129124177
37900 129126253
37901 129128049
37902 129128914
37903 129129928
37904 129132130
37905 129134297
37906 129135744
37907 129137613
37908 129140774
37909 129145102
37910 129155229
37911 129155691
37912 129156243
37913 129156248
37914 129159356
37915 129160017
37916 129160060
37917 129162164
37918 129162486
37919 129163395
37920 129166764
37921 129166779
37922 129166915
37923 129167778
37924 129168590
37925 129168595
37926 129170041
37927 129176165
37928 129176579
37929 129176682
37930 129176780
37931 129180305
37932 129185779
37933 129187864
37934 129193322
37935 129196655
37936 129198704
37937 129200957
37938 129202751
37939 129203790
37940 129210284
37941 129211923
37942 129213615
37943 129218822
37944 129222868
37945 129223800
37946 129224032
37947 129224125
37948 129224509
37949 129226222
37950 129227798
37951 129228008
37952 129232293
37953 129232571
37954 129233375
37955 129233493
37956 129234696
37957 129235286
37958 129235888
37959 129236554
37960 129237862
37961 12

KeyboardInterrupt: 